In [2]:
import os
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import box
from shutil import copy2


In [3]:
# %% Caminhos
base_folder = r'C:\temp\drones\saida'
output_classes_folder = r'C:\temp\drones\classes'

# Cria a pasta de saída principal para as classes
os.makedirs(output_classes_folder, exist_ok=True)

In [ ]:
# %% Processamento
# Itera por todas as subpastas no diretório base
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    # Verifica se é uma subpasta válida
    if os.path.isdir(folder_path):
        geojson_path = os.path.join(base_folder, f"{folder_name}.geojson")

        # Verifica se o arquivo GeoJSON correspondente existe
        if os.path.exists(geojson_path):
            # Lê e reprojeta o GeoJSON para EPSG:32723
            gdf = gpd.read_file(geojson_path)
            gdf = gdf.to_crs(epsg=32723)

            # Identifica os tipos únicos no GeoJSON e cria pastas de saída por classe
            types = gdf["tipo"].unique()
            for t in types:
                os.makedirs(os.path.join(output_classes_folder, t), exist_ok=True)

            # Itera pelos arquivos GeoTIFF na subpasta
            for image_file in os.listdir(folder_path):
                if image_file.endswith(".tif"):
                    image_path = os.path.join(folder_path, image_file)
                    with rasterio.open(image_path) as src:
                        # Obtém os limites do GeoTIFF
                        image_bounds = box(*src.bounds)

                        # Verifica interseção com os polígonos no GeoJSON
                        overlaps = gdf[gdf.geometry.intersects(image_bounds)]
                        if not overlaps.empty:
                            # Determina a classe predominante no corte
                            total_pixels = src.height * src.width
                            predominant_type = None
                            max_coverage = 0
                            for _, row in overlaps.iterrows():
                                # Gera a máscara para a interseção
                                mask = geometry_mask(
                                    [row.geometry],
                                    transform=src.transform,
                                    out_shape=(src.height, src.width),
                                    invert=True
                                )
                                covered_pixels = mask.sum()
                                coverage_ratio = covered_pixels / total_pixels
                                if coverage_ratio > max_coverage:
                                    max_coverage = coverage_ratio
                                    predominant_type = row["tipo"]

                            # Copia o arquivo para a pasta correspondente à classe
                            if predominant_type in ["areia", "restinga"]:
                                destination_folder = os.path.join(output_classes_folder, predominant_type)
                                copy2(image_path, os.path.join(destination_folder, image_file))
